### 基金的季度报告查找下载   
2021年4月19日


In [1]:
import requests
import random
from bs4 import BeautifulSoup as bs
import time
#import redis
import re
import json
from tqdm.notebook import tqdm 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                                'Chrome/78.0.3904.108 Safari/537.36'}

def downloadFILE_old(url,name):
    try:
        resp = requests.get(url=url,headers=headers,stream=True)
        content_size = int(resp.headers['Content-Length'])/1024
        with open(name, "wb") as f:
            #print("Pkg total size is:",content_size,'k,start...')
            print("文件%s的大小是：%.2fK,开始下载..." % (name,content_size))
            for data in tqdm(iterable=resp.iter_content(1024),total=content_size,unit='k',desc=name):
                f.write(data)
        print(name + "下载完成！")
    except Exception as e:
        print(name + "下载失败！", e)

In [36]:
## 读取基金页面的规模
def get_jjgm(jjcode):
    timeout=10
    url = f'http://fund.eastmoney.com/{jjcode}.html'
    try:
        page = requests.get(url, headers=header, timeout=timeout)
        page.encoding = 'utf-8'
        result = ('success', page.text)
    except (requests.exceptions.ConnectionError, requests.exceptions.Timeout, requests.exceptions.HTTPError):
        result = ('error', url)
    page_context = result[1]
    fund_scale = re.search(r'基金规模</a>：((?:\d+(?:\.\d{2}|)|--)亿元.*?)<', page_context)
    print(fund_scale.group(1))
get_jjgm("168104")    

0.73亿元（2021-03-31）


In [6]:
df_fund = pd.read_excel("300502基金持股.xls")

In [ ]:
fund_scale = re.search(r'基金规模</a>：((?:\d+(?:\.\d{2}|)|--)亿元.*?)<', page_context)

In [7]:
df_fund.code=df_fund.code.map(lambda x: "%06d"%x)
fund_list = df_fund.code.tolist()

In [8]:
#fund_list = df_fund.sort_values(by="s2004",ascending=False).head(40).code.tolist()
fund_list = df_fund.sort_values(by="s2004",ascending=False).code.tolist()

In [3]:
browser = webdriver.Chrome()
jjcode = "009062"
url = "http://fundf10.eastmoney.com/ccbdxq_%s_300502.html" % jjcode

In [4]:
def check_cg(jjcode,stockid):  #查找2020年12月31日到2021年1月31日两次季报对stockid的持股是否有增加
    url = "http://fundf10.eastmoney.com/ccbdxq_%s_300502.html" % jjcode
    browser.get(url)
    title_element = browser.find_elements_by_xpath("//h4[@class='title']")
    if len(title_element)>0:
        title = title_element[0].text
    target = browser.find_elements_by_xpath("//div[@id='cctable']")
    df = pd.read_html(target[0].get_attribute("outerHTML"))[0]
    if df.loc[0,"报告期"]=="2021-03-31" :
        s2004 = df.loc[df["报告期"]=="2020-12-31","持股数（万股）"].values[0]
        s2101 = df.loc[df["报告期"]=="2021-03-31","持股数（万股）"].values[0]
        print(f"基金 {title} 持有{stockid}的变动情况为：从{s2004}万股到今年一季报的{s2101}万股，增加了{round(s2101-s2004,2)}万股  ")
        #print(df)
    else:
        pass
        #print(f"基金{title}的一季报还没出")
check_cg("009970","300502")            

基金 财通内需增长12个月定开混合 (009970) 持有300502的变动情况为：从113.55万股到今年一季报的113.55万股，增加了0.0万股  


In [8]:
for i in fund_list:
    try:
        check_cg(i,"300502")
    except:
        pass

基金 银华同力精选混合 (009394) 持有300502的变动情况为：从262.03万股到今年一季报的419.0万股，增加了156.97万股  
基金 财通内需增长12个月定开混合 (009970) 持有300502的变动情况为：从113.55万股到今年一季报的113.55万股，增加了0.0万股  
基金 银华创业板两年定期开放混合 (161838) 持有300502的变动情况为：从76.98万股到今年一季报的132.88万股，增加了55.9万股  
基金 中邮信息产业灵活配置混合 (001227) 持有300502的变动情况为：从62.18万股到今年一季报的95.18万股，增加了33.0万股  
基金 富国中证500指数(LOF) (161017) 持有300502的变动情况为：从54.85万股到今年一季报的63.15万股，增加了8.3万股  
基金 财通科创主题3年封闭混合 (501085) 持有300502的变动情况为：从155.19万股到今年一季报的155.19万股，增加了0.0万股  
基金 银华成长先锋混合 (180020) 持有300502的变动情况为：从17.01万股到今年一季报的46.62万股，增加了29.61万股  
基金 华夏中证5G通信主题ETF (515050) 持有300502的变动情况为：从420.74万股到今年一季报的426.7万股，增加了5.96万股  
基金 南方信息创新混合A (007490) 持有300502的变动情况为：从769.68万股到今年一季报的592.11万股，增加了-177.57万股  
基金 南方多利增强债券C (202102) 持有300502的变动情况为：从3.79万股到今年一季报的3.79万股，增加了0.0万股  
基金 中邮创新优势灵活配置混合 (001275) 持有300502的变动情况为：从13.5万股到今年一季报的18.5万股，增加了5.0万股  
基金 南方养老2040三年持有混合(F (009572) 持有300502的变动情况为：从1.89万股到今年一季报的1.89万股，增加了0.0万股  
基金 建信中证1000指数增强A (006165) 持有300502的变动情况为：从1.02万股到今年一季报的2.08万股，增加了1.06万股  
基金 富国中证1000指数增强 (161039) 持有

基金 南方信息创新混合A (007490) 持有300502的变动情况为：从769.68万股到今年一季报的592.11万股，增加了-177.57万股  
基金 财通成长优选混合 (001480) 持有300502的变动情况为：从283.88万股到今年一季报的283.88万股，增加了0.0万股  
基金 财通价值动量混合 (720001) 持有300502的变动情况为：从283.88万股到今年一季报的283.88万股，增加了0.0万股  
基金 银华同力精选混合 (009394) 持有300502的变动情况为：从262.03万股到今年一季报的419.0万股，增加了156.97万股  
基金 财通科创主题3年封闭混合 (501085) 持有300502的变动情况为：从155.19万股到今年一季报的155.19万股，增加了0.0万股  
基金 财通内需增长12个月定开混合 (009970) 持有300502的变动情况为：从113.55万股到今年一季报的113.55万股，增加了0.0万股  
基金 银华创业板两年定期开放混合 (161838) 持有300502的变动情况为：从76.98万股到今年一季报的132.88万股，增加了55.9万股  
基金 中邮信息产业灵活配置混合 (001227) 持有300502的变动情况为：从62.18万股到今年一季报的95.18万股，增加了33.0万股  
基金 富国中证500指数(LOF) (161017) 持有300502的变动情况为：从54.85万股到今年一季报的63.15万股，增加了8.3万股  
基金 财通集成电路产业股票A (006502) 持有300502的变动情况为：从39.55万股到今年一季报的39.55万股，增加了0.0万股  
基金 财通多策略福鑫定开混合 (501046) 持有300502的变动情况为：从32.67万股到今年一季报的32.67万股，增加了0.0万股  
基金 银华成长先锋混合 (180020) 持有300502的变动情况为：从17.01万股到今年一季报的46.62万股，增加了29.61万股  
基金 中邮创新优势灵活配置混合 (001275) 持有300502的变动情况为：从13.5万股到今年一季报的18.5万股，增加了5.0万股  
基金 富国中证1000指数增强 (161039) 持有300502的变动情况为：从4.25万股到今年一季报的6.07万股，增加了1.82万股  
基金 南方多利增强债券C (202102) 持有300502的变动情况为：从3.79万股到今年一季报的3.79万股，增加了0.0万股  
基金 南方养老2040三年持有混合(F (009572) 持有300502的变动情况为：从1.89万股到今年一季报的1.89万股，增加了0.0万股  
基金 建信中证1000指数增强A (006165) 持有300502的变动情况为：从1.02万股到今年一季报的2.08万股，增加了1.06万股  
基金 红土创新沪深300增强A (006698) 持有300502的变动情况为：从0.07万股到今年一季报的0.43万股，增加了0.36万股  

In [146]:
#target = browser.find_elements_by_xpath("//div[@id='cctable']")
title = browser.find_elements_by_xpath("//h4[@class='title']")

In [149]:
title[0].text

'银华创业板两年定期开放混合 (161838)'

In [88]:
df = pd.read_html(target[0].get_attribute("outerHTML"))


In [108]:
temp = df[0].loc[0,"报告期"]

In [111]:
print(f"基金{temp}的一季报还没出")

基金f2020-12-31的一季报还没出


In [103]:
len(df[0][df[0]["报告期"]=="2020-12-31"])

1

In [66]:
def check_fund(fundcode):  # 看看基金是否有一季报
    url = "http://fundf10.eastmoney.com/jjgg_%s_3.html" % fundcode
    browser.get(url)
    #soup = bs(browser.page_source, "html.parser")
    target = browser.find_elements_by_xpath("//td/a[contains(text(),'2021年第1季度报告')]")  #找出含有季度报告的数据 表格里的A
    if len(target)>0:
        print(fundcode)

In [67]:
for i in fund_list:
    check_fund(i)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.128)


In [43]:


check_fund("005827")

005827


In [9]:
# 获取所有股票的名字，存入当地文件
 
def get_stock_names():
    """
    通过东方财富网上爬取股票的名称代码,并存入redis数据库和本地txt文档
    """
    #rds = redis.from_url('redis://:666666@192.168.3.98:6379', db=1, decode_responses=True)   # 连接redis db1
 
    url = "http://quote.eastmoney.com/stocklist.html"
    headers = {
            'Referer': 'http://quote.eastmoney.com/center/gridlist.html',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
        }
 
    response = requests.get(url, headers=headers).content.decode('utf-8')   # 网站编码为gbk 需要解码
    #print(response)
    soup = bs(response, 'lxml')
    #print(soup.text)
    all_ul = soup.find('table', id='table_wrapper-table').find_all('ul')   # 获取两个ul 标签数据
    with open('stock_names.txt', 'w+', encoding='utf-8') as f: 
        for ul in all_ul:
            all_a = ul.find_all('a')            # 获取ul 下的所有的a 标签
            for a in all_a:
                #rds.rpush('stock_names', a.text)       # a.text 为a标签中的text数据  rpush将数据右侧插入数据库
                print(a.text)
                f.write(a.text + '\n')
get_stock_names()                

In [7]:
url = "http://fundf10.eastmoney.com/jjgg_007490_3.html"
headers = {
        'Referer': 'http://quote.eastmoney.com/center/gridlist.html',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
    }

response = requests.get(url, headers=headers).content.decode('utf-8')   # 网站编码为gbk 需要解码
#print(response)
soup = bs(response, 'lxml')

In [10]:
with open('temp', 'w', encoding='utf-8') as f: 
    f.write(response)

In [11]:
driver = webdriver.Ie()
driver.get(url)

WebDriverException: Message: 'IEDriverServer.exe' executable needs to be in PATH. Please download from http://selenium-release.storage.googleapis.com/index.html and read up at https://github.com/SeleniumHQ/selenium/wiki/InternetExplorerDriver


In [35]:
#chrome_path = "E:\\Download\\ChromeDownload\\chromedriver_win32\\chromedriver.exe"
browser = webdriver.Chrome()

In [37]:
url = "http://fundf10.eastmoney.com/jjgg_%s_3.html" % "007490"
browser.get(url)
target = browser.find_elements_by_xpath("//td/a[contains(text(),'2021年1季度报告')]") 
if len(target)>0:
    print("007490")

In [34]:
 #driver.find_elements_by_xpath("//*[contains(text(), 'My Button')]")
 #target = browser.find_elements_by_xpath("//td/a[contains(text(),'季度报告')]/following-sibling::a")
browser.get(url)

NoSuchWindowException: Message: Unable to get browser


In [72]:
target[0].find_element_by_xpath("./following-sibling::a").get_attribute("href")  #找出下一个相邻的A

'http://pdf.dfcfw.com/pdf/H2_AN202101221453171343_1.pdf'

In [83]:
downloadFILE_old("http://pdf.dfcfw.com/pdf/H2_AN202101221453171343_1.pdf","1.pdf")

文件1.pdf的大小是：518.55K,开始下载...



1.pdf下载完成！


In [61]:
# 2021年4月20日
# 在使用fundcrawler.py生成相应的文件后，读取相应的基金文件进行排序
def p(x):
    if "-" in x:
        return 0
    else:
        return float(x.strip("%"))
df = pd.read_csv("FundCrawler/results/混合型.csv",engine="python")
for i in ["近1月","近3月","近6月","近1年","近3年","成立来"]:
    df[i]=df[i].map(p)

In [62]:
df["bj"]=df["近3月"]+df["近6月"]+df["近1年"]+df["近3年"]

In [63]:
df.sort_values(by="bj",ascending=False)

,基金名称,基金代码,基金规模,近1月,近3月,近6月,近1年,近3年,成立来,基金经理,任职时间,任期收益,总任职时间,bj
843,前海开源中国稀缺资产混合A,1679,30.35亿元（2020-12-31）,2.98,0.00,20.99,82.38,250.29,204.40,曲扬,5年又223天,197.10%,7年又157天,353.66
933,前海开源中国稀缺资产混合C,2079,11.94亿元（2020-12-31）,2.97,0.00,20.96,82.16,248.96,218.60,曲扬,5年又153天,210.90%,7年又157天,352.08
633,易方达新收益混合A,1216,37.58亿元（2021-03-31）,8.92,0.00,22.75,87.50,204.23,288.50,张清华,2年又102天,192.66%,7年又119天,314.48
222,农银研究精选混合,336,27.43亿元（2020-12-31）,5.59,0.00,21.63,117.53,172.31,252.86,赵诣,4年又30天,131.53%,4年又30天,311.47
2071,交银股息优化混合,4868,7.39亿元（2020-12-31）,6.49,3.07,25.38,106.58,176.29,187.26,韩威俊,3年又238天,184.02%,5年又91天,311.32
793,易方达新收益混合C,1217,18.64亿元（2021-03-31）,8.87,0.00,22.64,87.07,200.95,279.20,张清华,2年又102天,190.20%,7年又119天,310.66
135,前海联合泓鑫混合A,2780,6.35亿元（2020-12-31）,8.66,0.00,18.76,87.60,203.21,215.03,何杰,1年又214天,130.04%,3年又8天,309.57
308,长安裕泰混合C,5342,0.77亿元（2020-12-31）,5.85,0.00,25.40,82.56,193.77,176.00,徐小勇,2年又113天,213.70%,9年又200天,301.73
1641,中欧价值智选混合E,1887,0.53亿元（2020-12-31）,1.29,13.37,29.49,79.01,178.04,225.17,袁维德,325天,73.53%,4年又113天,299.91
1654,中欧价值智选混合A,166019,4.52亿元（2020-12-31）,1.30,13.37,29.50,79.04,177.19,432.97,袁维德,325天,73.54%,4年又113天,299.10
